In [ ]:
import os
import numpy as np
import pandas as pd

def load_eeg_data(data_dir):
    rows = []

    for condition in ['Healthy', 'AD']:
        for state in ['Eyes_open', 'Eyes_closed']:
            folder_path = os.path.join(data_dir, condition, state)
            if not os.path.exists(folder_path):
                print(f"Directory {folder_path} not found.")
                continue

            for paciente in os.listdir(folder_path):
                paciente_folder = os.path.join(folder_path, paciente)
                if os.path.isdir(paciente_folder):
                    for file in os.listdir(paciente_folder):
                        if file.endswith('.txt'):
                            file_path = os.path.join(paciente_folder, file)
                            try:
                                data = np.loadtxt(file_path)
                                rows.append({
                                    'Condition': condition,
                                    'State': state,
                                    'Patient': paciente,
                                    'File': file,
                                    'Signal': data
                                })
                            except Exception as e:
                                print(f"Error loading {file_path}: {e}")
    
    eeg_df = pd.DataFrame(rows)
    print("EEG data loaded successfully. Number of records:", len(eeg_df))
    return eeg_df

In [ ]:
# Étape 1 : Charger les données EEG
data_dir = "./EEG_data"
eeg_data_df = load_eeg_data(data_dir)

# Étape 2 : Nettoyage des données
def clean_data(df):
    # Supprimer les lignes avec des valeurs manquantes dans le signal
    df = df.dropna(subset=['Signal'])
    print("Valeurs manquantes supprimées. Nombre de lignes restantes :", len(df))

    # Filtrer les valeurs aberrantes : par exemple, en supprimant les signaux avec des amplitudes extrêmes
    df = df[df['Signal'].apply(lambda x: np.all(np.abs(x) < 1000))]  # Exemple de seuil
    print("Valeurs aberrantes filtrées. Nombre de lignes restantes :", len(df))

    # Normaliser les données si nécessaire
    df['Signal'] = df['Signal'].apply(lambda x: (x - np.mean(x)) / np.std(x))
    print("Les signaux ont été normalisés.")

    return df

# Appliquer le nettoyage des données
eeg_data_cleaned = clean_data(eeg_data_df)
